# Deep Learning Specialisation

## Hyperparameter tuning, regulatrisation and optimisation

### L2 regularisation, dropout, vanishing and exploding gradients, stadard scaling

### Bias variance

![bias-variance](./img/bias-variance.drawio.svg)

### Train, develop, test split

![Train develop test](./img/train-develop-test.drawio.svg)

- Decrease bias by making the model more complex, e.g. bigger network
- Decrease variance with more data, regularisation, dropout, new architecture
- The most common form of regularisation is L2 regularisation, so the cost function is
$$
\begin{align}
J &= -\frac{1}{m} \sum_{i=1}^my \log(\hat{y}^{i}) + (1-y^{i})\log(1-\hat{y}^{i}) + \frac{\lambda}{2m} \sum_{l=1}^{L}\|W^l\|^2 \\
J &= -\frac{1}{m} \sum_{i=1}^my \log(\hat{y}^{i}) + (1-y^{i})\log(1-\hat{y}^{i}) + \frac{\lambda}{2m} 
 \sum_{l=1}^{L}\sum_{i=1}^{n_{l}}\sum_{j=1}^{n_{l-1}}(W_{ij}^{[l]})^2 \\
J &= -\frac{1}{m}(Ylog(A)^T + (1-Y)log(1-A)^T) + \frac{\lambda}{2m}\text{trace}(W^TW)
\end{align}
$$
- In dropout each node is dropped out based on parameter `keep_prob`. When implementing we introduce $D$ a matrix of 1s and 0s then drop out nodes by multiplying
$$A^{[l]} = D^{[l]} * A^{[l]} / \text{Keep Probabiliy }$$
- Normalise inputs using strandard scaling to increase the speed of training a nueral network
- Vanishing and exploding gradients happend due to many multiplications of terms in back prop. To avoid this use good initialisation with $$W^{[l]} = np.random.rand(shape) * \sqrt(\frac{2}{n^{[l-1]}})$$




### L2 regularisation

In [14]:
from typing import Callable

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.special import expit

def relu(x):
    return x * (x > 0)

def relu_derivative(x):
    return 1. * (x > 0)

sigmoid = expit

def sigmoid_derivative(z):
    return sigmoid(z) * (1.0 - sigmoid(z))

def leaky_relu(x, leaky_constant:float = 0.01):
    return np.where(x > 0.0, x,  x * leaky_constantÏ)

def leaky_relu_derivative(x, leaky_constant=0.01):
    return np.where(x > 0, 1, leaky_constant)


def log_loss(A, Y, Ws, regularisation_lambda:float = 0):
    m = A.shape[1] # m is number samples
    cost = - (1/m) * ((Y @ np.log(A).T + (1 - Y) @ np.log(1 - A).T))
    if not regularisation_lambda:
        return cost
    return cost + (regularisation_lambda / (2 * m)) * sum(np.square(W).sum() for W in Ws.values())

def square_loss(A, Y, Ws, regularisation_lambda:float = 0):
    m = A.shape[1] # m is number samples
    cost = - (1/m) * ((A - Y) @ (A - Y).T)
    if not regularisation_lambda:
        return cost
    return cost + (regularisation_lambda / (2 * m)) * sum(np.square(W).sum() for W in Ws.values())

ACTIVATION_FUNCTIONS : dict[str, Callable] = {
    "relu": relu,
    "leaky_relu": leaky_relu,
    "sigmoid": expit,
}

ACTIVATION_FUNCTION_DERIVATIVES: dict[str, Callable] = {
    "relu": relu_derivative,
    "leaky_relu": leaky_relu_derivative,
    "sigmoid": sigmoid_derivative,
}

LOSS_FUNCTIONS: dict[str, Callable] = {
    "log_loss": log_loss,
    "square_loss": square_loss
}

In [17]:
import logging

logger = logging.getLogger()

class NeuralNetwork:
    layer_activations: dict[int, str]
    def __init__(
        self,
        layer_sizes: list[int],
        learning_rate: float = 0.5,
        layer_activations: dict[int, str] | None = None,
        regularisation_lambda: float = 0,
        cost_function: str = "log_loss",
        model_id: str = "",
    ):
        self.learning_rate = learning_rate
        self.layer_sizes = layer_sizes
        self.regularisation_lambda = regularisation_lambda
        self.L = len(layer_sizes) - 1
        self.m = layer_sizes[0]
        self.cost_function = cost_function
        self.model_id = model_id
        if layer_activations:
            self.layer_activations = layer_activations
        else:
            # This sets all hidden layers and the output layer to "sigmoid" by default.
            self.layer_activations = {l:"sigmoid" for l in range(1, self.L)} | {self.L:"sigmoid"}

    def initialise_weights(self) -> None:
        # This is using He initialisation. Try changing to * 0.01 and see the change in cost plot.
        self.Ws = {
            l:np.random.normal(size=(n_l, n_l_minus_1)) * np.sqrt(2 / n_l_minus_1)
            for (l, (n_l, n_l_minus_1)) in enumerate(zip(self.layer_sizes[1:], self.layer_sizes), start=1)
        }
        self.bs = {l:np.zeros((n_l, 1)) for l, n_l in enumerate(self.layer_sizes[1:], start=1)}
        logger.info("Weights initialised")
        logger.debug(f"{self.Ws=}")

    def forward(self, X, cache=False) -> None:
        Zs, As = {}, {0:X}
        for l in range(1, self.L + 1):
            Zs[l] = self.Ws[l] @ As[l-1] + self.bs[l]
            g = ACTIVATION_FUNCTIONS[self.layer_activations[l]]
            logger.debug(f"Applying {self.layer_activations[l]} in layer{l}")
            As[l] = g(Zs[l])
        if cache:
            self.Zs, self.As = Zs, As
        return As[self.L]

    def backward(self, Y) -> None:
        dZs = {self.L: self.As[self.L] - Y}
        m = self.As[0].shape[1]
        dWs, dbs = {}, {}
        for l in range(self.L, 0, -1):
            logger.debug(f"calculating dZ for layer_id {l}")
            if l != self.L:
                dZs[l] = self.Ws[l+1].T @ dZs[l+1] * \
                    ACTIVATION_FUNCTION_DERIVATIVES[self.layer_activations[l]](self.Zs[l])
                    # For sigmoid we could just use self.As[l] * (1 - self.As[l])
            dWs[l] = (1.0 / m) * dZs[l] @ self.As[l-1].T
            if self.regularisation_lambda:
                dWs[l] += ((self.regularisation_lambda / m) * self.Ws[l])
            dbs[l] = (1.0 / m) * np.sum(dZs[l], axis=1, keepdims=True)
        self.dWs, self.dbs = dWs, dbs

    def update_weights(self):
        for l in range(1, self.L + 1):
            self.Ws[l] -= self.learning_rate * self.dWs[l]
            self.bs[l] -= self.learning_rate * self.dbs[l]

    def train(self, X, Y, n_epochs=10, log_every=100, plot_cost=False, fig=None):
        import plotly.graph_objs as go
        from IPython.display import display, clear_output

        costs = []
        epochs = []

        if plot_cost:
            if fig is None:
                fig = go.FigureWidget()
                display(fig)
            fig.add_scatter(x=[], y=[], mode='lines+markers', name=f'{self.model_id}')
            fig.update_layout(title='Training Cost over Epochs', xaxis_title='Epoch', yaxis_title='Cost')

        for epoch in range(n_epochs):
            A = self.forward(X, cache=True)
            self.backward(Y)
            self.update_weights()
            cost = self.cost(A, Y)
            if epoch % log_every == 0:
                logger.info(f"Cost after epoch {epoch} = {cost}")
            if plot_cost:
                costs.append(cost.item()) # extract from (1, 1) array
                epochs.append(epoch)
                if epoch % 10 == 0:
                    with fig.batch_update():
                        fig.data[-1].x = epochs
                        fig.data[-1].y = costs

    def cost(self, A, Y):
        if self.cost_function == "log_loss":
            return log_loss(A, Y, self.Ws, self.regularisation_lambda)
        elif self.cost_function == "square_loss":
            return square_loss(A, Y, self.Ws, self.regularisation_lambda)
        else:
            raise Exception(f"Incorrect value for self.cost_function:= {self.cost_function}")

    def predict(self, X, return_probability=False):
        Y_hat = self.forward(X)
        if return_probability:
            return Y_hat
        return np.where(Y_hat>0.5, 1, 0)


In [18]:

def regularisation_example():

    X_train = pd.read_feather('../titanic/processed/X_train.feather').to_numpy().T
    y_train = pd.read_feather('../titanic/processed/y_train.feather').to_numpy().T
    X_test = pd.read_feather('../titanic/processed/X_test.feather').to_numpy().T
    y_test = pd.read_feather('../titanic/processed/y_test.feather').to_numpy().T

    # Define a simple neural network architecture
    layers = [30, 50, 20, 1]

    fig = go.FigureWidget()
    display(fig)

    # Fit with lambda = 0 (no regularization)
    nn_no_reg = NeuralNetwork(layers, regularisation_lambda=0, model_id="No regularisation")
    nn_no_reg.initialise_weights()
    nn_no_reg.train(X_train, y_train, n_epochs=400, log_every=500, plot_cost=True, fig=fig)
    train_Y_pred_no_reg = nn_no_reg.predict(X_train)
    test_Y_pred_no_reg = nn_no_reg.predict(X_test)

    # Fit with lambda = 0.2 (with regularization)
    nn_reg = NeuralNetwork(layers, regularisation_lambda=1, model_id="With Regularisation")
    nn_reg.initialise_weights()
    nn_reg.train(X_train, y_train, n_epochs=400, log_every=500, plot_cost=True, fig=fig)
    train_Y_pred_reg = nn_reg.predict(X_train)
    test_Y_pred_reg = nn_reg.predict(X_test)

    train_accuracy_no_reg = (train_Y_pred_no_reg == y_train).sum() / y_train.shape[1]
    test_accuracy_no_reg = (test_Y_pred_no_reg == y_test).sum() / y_test.shape[1]
    train_accuracy_reg = (train_Y_pred_reg == y_train).sum() / y_train.shape[1]
    test_accuracy_reg = (test_Y_pred_reg == y_test).sum() / y_test.shape[1]

    print(f"Accuracy on train with no regularisation: {train_accuracy_no_reg}")
    print(f"Accuracy on test with no regularisation: {test_accuracy_no_reg}")
    print(f"Accuracy on train with regularisation: {train_accuracy_reg}")
    print(f"Accuracy on test with regularisation: {test_accuracy_reg}")


regularisation_example()

FigureWidget({
    'data': [], 'layout': {'template': '...'}
})

Accuracy on train with no regularisation: 0.8328651685393258
Accuracy on test with no regularisation: 0.8044692737430168
Accuracy on train with regularisation: 0.827247191011236
Accuracy on test with regularisation: 0.8100558659217877


### Dropout

In [19]:
import logging

logger = logging.getLogger()

class NeuralNetwork:
    layer_activations: dict[int, str]
    def __init__(
        self,
        layer_sizes: list[int],
        learning_rate: float = 0.5,
        layer_activations: dict[int, str] | None = None,
        regularisation_lambda: float = 0.0,
        keep_prob: float = 1.0,
        cost_function: str = "log_loss",
        model_id: str = "",
    ):
        self.learning_rate = learning_rate
        self.layer_sizes = layer_sizes
        self.regularisation_lambda = regularisation_lambda
        self.L = len(layer_sizes) - 1
        self.m = layer_sizes[0]
        self.cost_function = cost_function
        self.keep_prob = keep_prob
        self.model_id = model_id
        if layer_activations:
            self.layer_activations = layer_activations
        else:
            # This sets all hidden layers and the output layer to "sigmoid" by default.
            self.layer_activations = {l:"sigmoid" for l in range(1, self.L)} | {self.L:"sigmoid"}

    def initialise_weights(self) -> None:
        # This is using He initialisation. Try changing to * 0.01 and see the change in cost plot.
        self.Ws = {
            l:np.random.normal(size=(n_l, n_l_minus_1)) * np.sqrt(2 / n_l_minus_1)
            for (l, (n_l, n_l_minus_1)) in enumerate(zip(self.layer_sizes[1:], self.layer_sizes), start=1)
        }
        self.bs = {l:np.zeros((n_l, 1)) for l, n_l in enumerate(self.layer_sizes[1:], start=1)}
        logger.info("Weights initialised")
        logger.debug(f"{self.Ws=}")

    def forward(self, X, cache=False) -> None:
        Zs, As, Ds = {}, {0:X}, {}
        for l in range(1, self.L + 1):
            Zs[l] = self.Ws[l] @ As[l-1] + self.bs[l]
            g = ACTIVATION_FUNCTIONS[self.layer_activations[l]]
            logger.debug(f"Applying {self.layer_activations[l]} in layer{l}")
            As[l] = g(Zs[l])
            # apply drop out
            if cache and l != self.L:
                Ds[l] = (np.random.uniform(size=As[l].shape) < self.keep_prob).astype(int) / self.keep_prob
                As[l] *= Ds[l]
        if cache:
            self.Zs, self.As, self.Ds = Zs, As, Ds
        return As[self.L]

    def backward(self, Y) -> None:
        dZs = {self.L: self.As[self.L] - Y}
        m = self.As[0].shape[1]
        dWs, dbs = {}, {}
        for l in range(self.L, 0, -1):
            logger.debug(f"calculating dZ for layer_id {l}")
            if l != self.L:
                # dj/dz = dj/da * da/dz
                dZs[l] = self.Ws[l+1].T @ dZs[l+1] * self.Ds[l] * \
                    ACTIVATION_FUNCTION_DERIVATIVES[self.layer_activations[l]](self.Zs[l])
                    # For sigmoid we could just use self.As[l] * (1 - self.As[l])
            dWs[l] = (1.0 / m) * dZs[l] @ self.As[l-1].T
            if self.regularisation_lambda and l != self.L:
                dWs[l] += ((self.regularisation_lambda / m) * self.Ws[l])
            dbs[l] = (1.0 / m) * np.sum(dZs[l], axis=1, keepdims=True)
        self.dWs, self.dbs = dWs, dbs

    def update_weights(self):
        for l in range(1, self.L + 1):
            self.Ws[l] -= self.learning_rate * self.dWs[l]
            self.bs[l] -= self.learning_rate * self.dbs[l]

    def train(self, X, Y, n_epochs=10, log_every=100, plot_cost=False, fig=None):
        import plotly.graph_objs as go
        from IPython.display import display, clear_output

        costs = []
        epochs = []

        if plot_cost:
            if fig is None:
                fig = go.FigureWidget()
                display(fig)
            fig.add_scatter(x=[], y=[], mode='lines+markers', name=self.model_id)
            fig.update_layout(title='Training Cost over Epochs', xaxis_title='Epoch', yaxis_title='Cost')

        for epoch in range(n_epochs):
            A = self.forward(X, cache=True)
            self.backward(Y)
            self.update_weights()
            cost = self.cost(A, Y).item()
            if epoch % log_every == 0:
                logger.info(f"Cost after epoch {epoch} = {cost}")
            logger.info(f"Cost after epoch {epoch} = {cost}")
            if plot_cost:
                if np.isnan(cost):
                    cost = 10
                costs.append(cost)
                epochs.append(epoch)
                if epoch % 10 == 0:
                    with fig.batch_update():
                        fig.data[-1].x = epochs
                        fig.data[-1].y = costs

    def cost(self, A, Y):
        if self.cost_function == "log_loss":
            return log_loss(A, Y, self.Ws, self.regularisation_lambda)
        elif self.cost_function == "square_loss":
            return square_loss(A, Y, self.Ws, self.regularisation_lambda)
        else:
            raise Exception(f"Incorrect value for self.cost_function:= {self.cost_function}")

    def predict(self, X, return_probability=False):
        Y_hat = self.forward(X)
        if return_probability:
            return Y_hat
        return np.where(Y_hat>0.5, 1, 0)


In [20]:
def dropout_example():

    X_train = pd.read_feather('../titanic/processed/X_train.feather').to_numpy().T
    y_train = pd.read_feather('../titanic/processed/y_train.feather').to_numpy().T
    X_test = pd.read_feather('../titanic/processed/X_test.feather').to_numpy().T
    y_test = pd.read_feather('../titanic/processed/y_test.feather').to_numpy().T

    # Define a simple neural network architecture
    layers = [30, 50, 20, 1]

    fig = go.FigureWidget()
    display(fig)

    # Fit with lambda = 0 (no regularization)
    nn_no_reg = NeuralNetwork(layers, keep_prob=1, model_id="No dropout")
    nn_no_reg.initialise_weights()
    nn_no_reg.train(X_train, y_train, n_epochs=500, log_every=500, plot_cost=True, fig=fig)
    train_Y_pred_no_reg = nn_no_reg.predict(X_train)
    test_Y_pred_no_reg = nn_no_reg.predict(X_test)

    # Fit with lambda = 0.2 (with regularization)
    nn_reg = NeuralNetwork(layers, keep_prob=0.8, model_id="20% drop out")
    nn_reg.initialise_weights()
    nn_reg.train(X_train, y_train, n_epochs=500, log_every=50, plot_cost=True, fig=fig)
    train_Y_pred_reg = nn_reg.predict(X_train)
    test_Y_pred_reg = nn_reg.predict(X_test)

    train_accuracy_no_reg = (train_Y_pred_no_reg == y_train).sum() / y_train.shape[1]
    test_accuracy_no_reg = (test_Y_pred_no_reg == y_test).sum() / y_test.shape[1]
    train_accuracy_reg = (train_Y_pred_reg == y_train).sum() / y_train.shape[1]
    test_accuracy_reg = (test_Y_pred_reg == y_test).sum() / y_test.shape[1]

    print(f"Accuracy on train with no regularisation: {train_accuracy_no_reg}")
    print(f"Accuracy on test with no regularisation: {test_accuracy_no_reg}")
    print(f"Accuracy on train with regularisation: {train_accuracy_reg}")
    print(f"Accuracy on test with regularisation: {test_accuracy_reg}")


dropout_example()

FigureWidget({
    'data': [], 'layout': {'template': '...'}
})

Accuracy on train with no regularisation: 0.8384831460674157
Accuracy on test with no regularisation: 0.8044692737430168
Accuracy on train with regularisation: 0.824438202247191
Accuracy on test with regularisation: 0.8100558659217877


### Standard scaling


In [21]:
from typing import Optional
from numpy.typing import NDArray
import pandas as pd
import plotly.graph_objects as go

class StandardScaler:

    def __init__(self):
        self.means: Optional[NDArray] = None
        self.stds: Optional[NDArray] = None
        self.is_fitted: bool = False

    def fit_transform(self, X) -> NDArray:
        self.means = X.mean(axis=1, keepdims=True)
        self.stds = X.std(axis=1, keepdims=True)
        return (X - self.means) / self.stds

    def transform(self, X) -> NDArray:
        return (X - self.means) / self.stds

    def fit(self, X: NDArray) -> None:
        self.means = X.mean(axis=1, keepdims=True)
        self.stds = X.std(axis=1, keepdims=True)
        self.is_fitted = True

def standard_scaling_example():
    X_train = pd.read_feather('../titanic/processed/X_train.feather').to_numpy().T
    y_train = pd.read_feather('../titanic/processed/y_train.feather').to_numpy().T
    X_test = pd.read_feather('../titanic/processed/X_test.feather').to_numpy().T
    y_test = pd.read_feather('../titanic/processed/y_test.feather').to_numpy().T

    scaler = StandardScaler()
    scaler.fit_transform(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    layers = [30, 50, 20, 1]

    nn = NeuralNetwork(layers, model_id="unscaled data")
    nn_scaled = NeuralNetwork(layers, model_id="scaled data")
    nn.initialise_weights()
    nn_scaled.initialise_weights()

    fig = go.FigureWidget()
    display(fig)
    nn.train(X_train, y_train, n_epochs=400, log_every=500, plot_cost=True, fig=fig)
    nn_scaled.train(X_train_scaled, y_train, n_epochs=400, log_every=500, plot_cost=True, fig=fig)

    train_Y_pred = nn.predict(X_train)
    test_Y_pred = nn.predict(X_test)
    train_Y_pred_scaled = nn_scaled.predict(X_train_scaled)
    test_Y_pred_scaled = nn_scaled.predict(X_test_scaled)

    train_accuracy = (train_Y_pred == y_train).sum() / y_train.shape[1]
    test_accuracy = (test_Y_pred == y_test).sum() / y_test.shape[1]
    train_accuracy_scaled = (train_Y_pred_scaled == y_train).sum() / y_train.shape[1]
    test_accuracy_scaled = (test_Y_pred_scaled == y_test).sum() / y_test.shape[1]

    print(f"Accuracy on train: {train_accuracy}")
    print(f"Accuracy on test: {test_accuracy}")
    print(f"Accuracy on train with scaled data: {train_accuracy_scaled}")
    print(f"Accuracy on test with scaled data: {test_accuracy_scaled}")

standard_scaling_example()

FigureWidget({
    'data': [], 'layout': {'template': '...'}
})

Accuracy on train: 0.8328651685393258
Accuracy on test: 0.8156424581005587
Accuracy on train with scaled data: 0.8455056179775281
Accuracy on test with scaled data: 0.8044692737430168


### Mini-batch training, Momentum, RMSprop and Adam

 - In mini-batch gradient descent, we use a subset of the training data to update the weights.
 - In batch gradient descent, we use all the training data to update the weights.
 - In stochastic gradient descent, we use a single training example to update the weights.
 - The momentum optimisation algorithm uses a moving average of the gradients to update the weights. Momentum helps by enabling optimizers to overcome local minima and saddle points, reduce oscillations in narrow valleys for a smoother path, and speed up learning by maintaining a consistent direction through flat or consistent gradient regions.
 $$
 \begin{align*}
 v_t &= \beta_1 v_{t-1} + (1 - \beta_1) \nabla J(w) \\
 w &= w - \alpha v_t
 \end{align*}
 $$ where $v_t$ is the moving average of the gradients, $\beta_1$ is the momentum coefficient, $\nabla J(w)$ is the gradient of the cost function $J(w)$ with respect to the weights $w$, and $\alpha$ is the learning rate.
 - The RMSprop optimisation algorithm maintains a moving average of the squared gradients, it then updates the weights using the gradients divided by the square root of the moving average. RMSprop adaptively adjusts learning rates: it increases them for parameters with small gradients (in flat areas) and decreases them for those with large gradients (in steep or oscillating areas), balancing updates. This method also normalizes gradient magnitudes, preventing them from becoming too small (vanishing) or too large (exploding), which leads to more stable and efficient training.
 $$
 \begin{align*}
 s_t &= \beta_2 s_{t-1} + (1 - \beta_2) (\nabla J(w))^2 \\
 w &= w - \alpha \frac{\nabla J(w)}{\sqrt{s_t + \epsilon}}
 \end{align*}
 $$ where $s_t$ is the moving average of the squared gradients, $\beta_2$ is the RMSprop coefficient, $\nabla J(w)$ is the gradient of the cost function $J(w)$ with respect to the weights $w$, and $\alpha$ is the learning rate.
 - The adam algorihtm combines the momentum and RMSprop optimisation algorithms. It uses a moving average of the gradients scaled by a moving average of the squared gradients. Adam also uses a bias correction term to adjust the moving averages.
 $$
 \begin{align*}
 v_t &= \beta_1 v_{t-1} + (1 - \beta_1) \nabla J(w) \\
 s_t &= \beta_2 s_{t-1} + (1 - \beta_2) (\nabla J(w))^2 \\
 \hat{v_t} &= \frac{v_t}{1 - \beta_1^t} \\
 \hat{s_t} &= \frac{s_t}{1 - \beta_2^t} \\
 w &= w - \alpha \frac{\hat{v_t}}{\sqrt{\hat{s_t} + \epsilon}}
 \end{align*}
 $$ where $v_t$ is the moving average of the gradients, $s_t$ is the moving average of the squared gradients, $\beta_1$ and $\beta_2$ are the momentum and RMSprop coefficients, $\nabla J(w)$ is the gradient of the cost function $J(w)$ with respect to the weights $w$, and $\alpha$ is the learning rate.


In [23]:
import plotly.graph_objs as go
from IPython.display import display
import logging

logger = logging.getLogger()


class NeuralNetwork:
    layer_activations: dict[int, str]

    def __init__(
        self,
        layer_sizes: list[int],
        layer_activations: dict[int, str] | None = None,
        regularisation_lambda: float = 0.0,
        keep_prob: float = 1.0,
        cost_function: str = "log_loss",
        model_id: str = "",
    ):
        self.layer_sizes = layer_sizes
        self.regularisation_lambda = regularisation_lambda
        self.L = len(layer_sizes) - 1
        self.m = layer_sizes[0]
        self.cost_function = cost_function
        self.keep_prob = keep_prob
        self.model_id = model_id
        if layer_activations:
            self.layer_activations = layer_activations
        else:
            # This sets all hidden layers and the output layer to "sigmoid" by default.
            self.layer_activations = {l: "sigmoid" for l in range(1, self.L)} | {
                self.L: "sigmoid"
            }

    def initialise_weights(self) -> None:
        # This is using He initialisation. Try changing to * 0.01 and see the change in cost plot.
        self.params = {}
        for l, (n_l, n_l_minus_1) in enumerate(
            zip(self.layer_sizes[1:], self.layer_sizes), start=1
        ):
            self.params[f"W{l}"] = np.random.normal(size=(n_l, n_l_minus_1)) * np.sqrt(
                2 / n_l_minus_1
            )
            self.params[f"b{l}"] = np.zeros((n_l, 1))
        logger.info("Weights initialised")
        logger.debug(f"{self.params=}")

    def forward(self, X, cache=False) -> None:
        Zs, As, Ds = {}, {0: X}, {}
        for l in range(1, self.L + 1):
            W = self.params[f"W{l}"]
            b = self.params[f"b{l}"]
            Zs[l] = W @ As[l - 1] + b
            g = ACTIVATION_FUNCTIONS[self.layer_activations[l]]
            logger.debug(f"Applying {self.layer_activations[l]} in layer{l}")
            As[l] = g(Zs[l])
            # apply drop out but not on the output layer
            if cache and l != self.L:
                Ds[l] = (np.random.uniform(size=As[l].shape) < self.keep_prob).astype(
                    int
                ) / self.keep_prob
                As[l] *= Ds[l]
        if cache:
            self.Zs, self.As, self.Ds = Zs, As, Ds
        return As[self.L]

    def backward(self, Y):
        dZs = {self.L: self.As[self.L] - Y}
        m = self.As[0].shape[1]
        grads = {}
        for l in range(self.L, 0, -1):
            logger.debug(f"calculating dZ for layer_id {l}")
            if l != self.L:
                W_l_plus_1 = self.params[f"W{l+1}"]
                dZs[l] = (
                    W_l_plus_1.T
                    @ dZs[l + 1]
                    * self.Ds[l]
                    * ACTIVATION_FUNCTION_DERIVATIVES[self.layer_activations[l]](
                        self.Zs[l]
                    )
                )
            grads[f"dW{l}"] = (1.0 / m) * dZs[l] @ self.As[l - 1].T
            if self.regularisation_lambda and l != self.L:
                grads[f"dW{l}"] += (self.regularisation_lambda / m) * self.params[
                    f"W{l}"
                ]
            grads[f"db{l}"] = (1.0 / m) * np.sum(dZs[l], axis=1, keepdims=True)
        return grads

    def cost(self, A, Y) -> float:
        # For cost, we need to pass the weights. We'll extract them from self.params.
        Ws = {l: self.params[f"W{l}"] for l in range(1, self.L + 1)}
        if self.cost_function == "log_loss":
            return log_loss(A, Y, Ws, self.regularisation_lambda).item()
        elif self.cost_function == "square_loss":
            return square_loss(A, Y, Ws, self.regularisation_lambda).item()
        else:
            raise Exception(
                f"Incorrect value for self.cost_function:= {self.cost_function}"
            )

    def predict(self, X, return_probability=False):
        Y_hat = self.forward(X)
        if return_probability:
            return Y_hat
        return np.where(Y_hat > 0.5, 1, 0)


class Optimizer:
    """This implements gradient descent. With optional mini-batching."""

    def __init__(self, learning_rate: float = 0.1, batch_size: int | None = None):
        self.learning_rate = learning_rate
        self.batch_size = batch_size

    def update_model_params(self, model, grads, training_iteration):
        """Update model parameters using grads returned from backward."""
        for param_key in model.params:
            model.params[param_key] -= self.learning_rate * grads[f"d{param_key}"]

    def train(
        self,
        model,
        X,
        Y,
        n_epochs=1000,
        log_every: int | None = None,
        plot_cost=False,
        fig=None,
        plot_every=10,
    ):
        costs, epochs = [], []

        if plot_cost:
            if fig is None:
                fig = go.FigureWidget()
                display(fig)
            fig.add_scatter(x=[], y=[], mode="lines+markers", name=model.model_id)
            fig.update_layout(
                title="Training Cost over Epochs",
                xaxis_title="Epoch",
                yaxis_title="Cost",
            )

        m = X.shape[1]
        batch_size = self.batch_size if self.batch_size is not None else m

        training_iteration = 1
        for epoch in range(n_epochs):
            for i in range(0, m, batch_size):
                X_batch = X[:, i : i + batch_size]
                Y_batch = Y[:, i : i + batch_size]

                # Forward pass
                A = model.forward(X_batch, cache=True)
                grads = model.backward(Y_batch)
                self.update_model_params(model, grads, training_iteration)
                training_iteration += 1

            # Compute cost on the whole dataset after epoch
            A_full = model.forward(X, cache=False)
            cost = model.cost(A_full, Y)
            if log_every and epoch % log_every == 0:
                logger.info(f"Cost after epoch {epoch} = {cost}")
            if plot_cost and fig is not None:
                costs.append(cost)
                epochs.append(epoch + 1)
                if epoch % plot_every == 0:
                    with fig.batch_update():
                        fig.data[-1].x = epochs  # type: ignore
                        fig.data[-1].y = costs  # type: ignore


class MomentumOptimizer(Optimizer):
    """Implements momentum optimizer.

    The update rule is:
    v_t = beta * v_{t-1} + grad_t
    param_t = param_{t-1} - learning_rate * v_t

    Where:
    - S_t is the second moment of the gradient
    - param_t is the parameter
    - grad_t is the gradient of the cost function with respect to the parameter
    """

    def __init__(
        self,
        learning_rate: float = 0.1,
        batch_size: int | None = None,
        beta: float = 0.9,
    ):
        super().__init__(learning_rate, batch_size)
        self.beta = beta
        self.cache = {}

    def update_model_params(self, model, grads, training_iteration):
        for param_key in model.params:
            if param_key not in self.cache:
                self.cache[param_key] = np.zeros_like(model.params[param_key])
            self.cache[param_key] = (
                self.beta * self.cache[param_key]
                + (1 - self.beta) * grads[f"d{param_key}"]
            )
            model.params[param_key] -= self.learning_rate * self.cache[param_key]


class RMSPropOptimizer(Optimizer):
    """Implements RMSProp optimizer.

    Here we track the exponentially weighted average of the squared gradients (second moment).

    The update rule is:
    S_t = beta * S_{t-1} + (1 - beta) * (grad_t)^2
    param_t = param_{t-1} - learning_rate * grad_t / (sqrt(S_t) + epsilon)
    """

    def __init__(
        self,
        learning_rate: float = 0.1,
        batch_size: int | None = None,
        beta: float = 0.9,
        epsilon: float = 1e-8,
    ):
        super().__init__(learning_rate, batch_size)
        self.beta = beta
        self.epsilon = epsilon  # to avoid division by zero
        self.s_cache = {}

    def update_model_params(self, model, grads, training_iteration):
        # Standard RMSProp does NOT use bias correction (unlike Adam)
        for param_key in model.params:
            if param_key not in self.s_cache:
                self.s_cache[param_key] = np.zeros_like(model.params[param_key])
            self.s_cache[param_key] = self.beta * self.s_cache[param_key] + (
                1 - self.beta
            ) * (grads[f"d{param_key}"] ** 2)
            model.params[param_key] -= (
                self.learning_rate
                * grads[f"d{param_key}"]
                / (np.sqrt(self.s_cache[param_key]) + self.epsilon)
            )


class ADAMOptimizer(Optimizer):
    """Implements ADAM optimizer with learning rate decay.

    The update rule is:
    v_t = beta_1 * v_{t-1} + (1 - beta_1) * grad
    s_t = beta_2 * s_{t-1} + (1 - beta_2) * (grad_t)^2
    v_t_corrected = v_t / (1 - beta_1^t)
    s_t_corrected = s_t / (1 - beta_2^t)
    param_t = param_{t-1} - learning_rate * v_t_corrected / (sqrt(s_t_corrected) + epsilon)

    """

    def __init__(
        self,
        learning_rate: float = 0.1,
        batch_size: int | None = None,
        beta_1: float = 0.9,
        beta_2: float = 0.999,
        epsilon: float = 1e-8,
        learning_rate_decay: float = 0,
    ):
        super().__init__(learning_rate, batch_size)
        self.beta_1 = beta_1  # for the momentum
        self.beta_2 = beta_2  # for the second moment (RMSProp)
        self.epsilon = epsilon  # to avoid division by zero
        self.learning_rate_decay = learning_rate_decay
        self.v_cache = {}
        self.s_cache = {}

    def update_model_params(self, model, grads, training_iteration):
        learning_rate = self.learning_rate * (
            1 / (1 + self.learning_rate_decay * training_iteration)
        )
        for param_key in model.params:
            if param_key not in self.v_cache:
                self.v_cache[param_key] = np.zeros_like(model.params[param_key])
                self.s_cache[param_key] = np.zeros_like(model.params[param_key])
            self.v_cache[param_key] = (
                self.beta_1 * self.v_cache[param_key]
                + (1 - self.beta_1) * grads[f"d{param_key}"]
            )
            self.s_cache[param_key] = (
                self.beta_2 * self.s_cache[param_key]
                + (1 - self.beta_2) * grads[f"d{param_key}"] ** 2
            )
            v_t_corrected = self.v_cache[param_key] / (
                1 - self.beta_1**training_iteration
            )
            s_t_corrected = self.s_cache[param_key] / (
                1 - self.beta_2**training_iteration
            )

            model.params[param_key] -= (
                learning_rate * v_t_corrected / (np.sqrt(s_t_corrected) + self.epsilon)
            )


def optimizer_example():
    X_train = pd.read_feather("../titanic/processed/X_train.feather").to_numpy().T
    y_train = pd.read_feather("../titanic/processed/y_train.feather").to_numpy().T
    X_test = pd.read_feather("../titanic/processed/X_test.feather").to_numpy().T
    y_test = pd.read_feather("../titanic/processed/y_test.feather").to_numpy().T

    layers = [30, 50, 20, 1]
    nn = NeuralNetwork(layers, model_id="Using an optimizer")
    nn_mini_batch = NeuralNetwork(layers, model_id="Using a mini-batch optimizer")
    nn_momentum = NeuralNetwork(layers, model_id="Using a momentum optimizer")
    nn_rmsprop = NeuralNetwork(layers, model_id="Using a RMSProp optimizer")
    nn_adam = NeuralNetwork(layers, model_id="Using a ADAM optimizer")
    nn_adam_decay = NeuralNetwork(
        layers, model_id="Using a ADAM optimizer with learning rate decay"
    )

    nn.initialise_weights()
    nn_mini_batch.initialise_weights()
    nn_momentum.initialise_weights()
    nn_rmsprop.initialise_weights()
    nn_adam.initialise_weights()
    nn_adam_decay.initialise_weights()

    optimizer = Optimizer(learning_rate=0.5)
    mini_batch_optimizer = Optimizer(batch_size=128)
    momentum_optimizer = MomentumOptimizer(batch_size=128)
    rmsprop_optimizer = RMSPropOptimizer(batch_size=128)
    adam_optimizer = ADAMOptimizer(batch_size=128)
    adam_decay_optimizer = ADAMOptimizer(batch_size=128, learning_rate_decay=0.5)

    models = [nn, nn_mini_batch, nn_momentum, nn_rmsprop, nn_adam, nn_adam_decay]
    optimizers = [
        optimizer,
        mini_batch_optimizer,
        momentum_optimizer,
        rmsprop_optimizer,
        adam_optimizer,
        adam_decay_optimizer,
    ]
    names = [
        "Gradient Descent",
        "Mini-batch",
        "Momentum",
        "RMSProp",
        "ADAM",
        "ADAM with learning rate decay",
    ]

    fig = go.FigureWidget()
    display(fig)

    from time import perf_counter

    for model, optimizer, name in zip(models, optimizers, names):
        start_time = perf_counter()
        optimizer.train(model, X_train, y_train, n_epochs=500, plot_cost=True, fig=fig)
        end_time = perf_counter()
        time = end_time - start_time
        print(f"\n Optimiser: {name}")
        print(f"{name} training time: {time:.4f} seconds")
        train_Y_pred = model.predict(X_train)
        test_Y_pred = model.predict(X_test)
        train_accuracy = (train_Y_pred == y_train).sum() / y_train.shape[1]
        test_accuracy = (test_Y_pred == y_test).sum() / y_test.shape[1]
        print(f"Accuracy on train: {train_accuracy}")
        print(f"Accuracy on test: {test_accuracy}")


optimizer_example()

FigureWidget({
    'data': [], 'layout': {'template': '...'}
})


 Optimiser: Gradient Descent
Gradient Descent training time: 3.1561 seconds
Accuracy on train: 0.8356741573033708
Accuracy on test: 0.8044692737430168

 Optimiser: Mini-batch
Mini-batch training time: 3.6310 seconds
Accuracy on train: 0.8356741573033708
Accuracy on test: 0.8100558659217877

 Optimiser: Momentum
Momentum training time: 4.1299 seconds
Accuracy on train: 0.8356741573033708
Accuracy on test: 0.8044692737430168


/var/folders/0f/7sr7n0x97jqbhl4pqwv79vdw0000gn/T/ipykernel_10477/328717897.py:28: RuntimeWarning:

divide by zero encountered in log

/var/folders/0f/7sr7n0x97jqbhl4pqwv79vdw0000gn/T/ipykernel_10477/328717897.py:28: RuntimeWarning:

invalid value encountered in matmul




 Optimiser: RMSProp
RMSProp training time: 4.7266 seconds
Accuracy on train: 0.9185393258426966
Accuracy on test: 0.7932960893854749

 Optimiser: ADAM
ADAM training time: 5.1561 seconds
Accuracy on train: 0.9410112359550562
Accuracy on test: 0.8212290502793296

 Optimiser: ADAM with learning rate decay
ADAM with learning rate decay training time: 5.5480 seconds
Accuracy on train: 0.8469101123595506
Accuracy on test: 0.8212290502793296


### Softmax regression



- In the case when there are C classes we want the model to output a probability distribution over the C classes. To acheive this we use C units in the output layer and apply the softmax function to the output of the last layer.
$$
 \begin{align*}
 \hat{y} &= \text{softmax}(z) \\
 \hat{y}_j &= \frac{e^{z_j}}{\sum_{c=1}^{C} e^{z_c}}
 \end{align*}
 $$

In [8]:
from typing import Callable
import logging

import seaborn as sns
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.special import expit
from IPython.display import display


logger = logging.getLogger()


def relu(x):
    return x * (x > 0)


def relu_derivative(x):
    return 1.0 * (x > 0)


sigmoid = expit


def sigmoid_derivative(z):
    return sigmoid(z) * (1.0 - sigmoid(z))


def leaky_relu(x, leaky_constant: float = 0.01):
    return np.where(x > 0.0, x, x * leaky_constant)


def leaky_relu_derivative(x, leaky_constant=0.01):
    return np.where(x > 0, 1, leaky_constant)


def log_loss(A, Y, Ws, regularisation_lambda: float = 0):
    m = A.shape[1]  # m is number samples
    cost = -(1 / m) * (Y @ np.log(A).T + (1 - Y) @ np.log(1 - A).T)
    if not regularisation_lambda:
        return cost
    return cost + (regularisation_lambda / (2 * m)) * sum(
        np.square(W).sum() for W in Ws.values()
    )


def log_loss(A, Y, Ws, regularisation_lambda: float = 0):
    m = A.shape[1]  # m is number of samples
    # Clip values to avoid log(0)
    epsilon = 1e-15
    A = np.clip(A, epsilon, 1 - epsilon)

    # For multi-class (softmax), we use categorical cross-entropy
    if A.shape[0] > 1:  # If output dimension > 1, assume multi-class
        # Categorical cross-entropy: -1/m * sum(y_i * log(a_i))
        cost = -np.sum(Y * np.log(A)) / m
    else:  # Binary classification case
        cost = -(1 / m) * (Y @ np.log(A).T + (1 - Y) @ np.log(1 - A).T)

    # Add L2 regularization if specified
    if regularisation_lambda:
        l2_cost = (regularisation_lambda / (2 * m)) * sum(
            np.square(W).sum() for W in Ws.values()
        )
        cost += l2_cost

    return np.array([[cost]])


def square_loss(A, Y, Ws, regularisation_lambda: float = 0):
    m = A.shape[1]  # m is number samples
    cost = -(1 / m) * ((A - Y) @ (A - Y).T)
    if not regularisation_lambda:
        return cost
    return cost + (regularisation_lambda / (2 * m)) * sum(
        np.square(W).sum() for W in Ws.values()
    )


def softmax(z):
    # if Z has shape (C, m) then we want to return a matrix of shape (C, m)
    # Note the softmax is applied for each column of Z
    stable_z = z - np.max(z, axis=0, keepdims=True)
    exponents = np.exp(stable_z)
    probabilities = exponents / np.sum(exponents, axis=0, keepdims=True)
    return probabilities


ACTIVATION_FUNCTIONS: dict[str, Callable] = {
    "relu": relu,
    "leaky_relu": leaky_relu,
    "sigmoid": expit,
    "softmax": softmax,
}

ACTIVATION_FUNCTION_DERIVATIVES: dict[str, Callable] = {
    "relu": relu_derivative,
    "leaky_relu": leaky_relu_derivative,
    "sigmoid": sigmoid_derivative,
}

LOSS_FUNCTIONS: dict[str, Callable] = {"log_loss": log_loss, "square_loss": square_loss}


class NeuralNetwork:
    """This implements a simple feedforward neural network.

    This additionally allows for dropout and regularisation and
    multi class output via the softmax function.
    """

    layer_activations: dict[int, str]

    def __init__(
        self,
        layer_sizes: list[int],
        layer_activations: dict[int, str] | None = None,
        regularisation_lambda: float = 0.0,
        keep_prob: float = 1.0,
        cost_function: str = "log_loss",
        model_id: str = "",
    ):
        self.layer_sizes = layer_sizes
        self.regularisation_lambda = regularisation_lambda
        self.L = len(layer_sizes) - 1
        self.m = layer_sizes[0]
        self.cost_function = cost_function
        self.keep_prob = keep_prob
        self.model_id = model_id
        if layer_activations:
            self.layer_activations = layer_activations
        else:
            # This sets all hidden layers and the output layer to "sigmoid" by default.
            self.layer_activations = {l: "sigmoid" for l in range(1, self.L)} | {
                self.L: "sigmoid"
            }

    def initialise_weights(self) -> None:
        # This is using He initialisation. Try changing to * 0.01 and see the change in cost plot.
        self.params = {}
        for l, (n_l, n_l_minus_1) in enumerate(
            zip(self.layer_sizes[1:], self.layer_sizes), start=1
        ):
            self.params[f"W{l}"] = np.random.normal(size=(n_l, n_l_minus_1)) * np.sqrt(
                2 / n_l_minus_1
            )
            self.params[f"b{l}"] = np.zeros((n_l, 1))
        logger.info("Weights initialised")
        logger.debug(f"{self.params=}")

    def forward(self, X, cache=False) -> None:
        Zs, As, Ds = {}, {0: X}, {}
        for l in range(1, self.L + 1):
            W = self.params[f"W{l}"]
            b = self.params[f"b{l}"]
            Zs[l] = W @ As[l - 1] + b
            g = ACTIVATION_FUNCTIONS[self.layer_activations[l]]
            logger.debug(f"Applying {self.layer_activations[l]} in layer{l}")
            As[l] = g(Zs[l])
            # apply drop out but not on the output layer
            if cache and l != self.L:
                Ds[l] = (np.random.uniform(size=As[l].shape) < self.keep_prob).astype(
                    int
                ) / self.keep_prob
                As[l] *= Ds[l]
        if cache:
            self.Zs, self.As, self.Ds = Zs, As, Ds
        return As[self.L]

    def backward(self, Y):
        dZs = {self.L: self.As[self.L] - Y}
        m = self.As[0].shape[1]
        grads = {}
        for l in range(self.L, 0, -1):
            logger.debug(f"calculating dZ for layer_id {l}")
            if l != self.L:
                W_l_plus_1 = self.params[f"W{l + 1}"]
                dZs[l] = (
                    W_l_plus_1.T
                    @ dZs[l + 1]
                    * self.Ds[l]
                    * ACTIVATION_FUNCTION_DERIVATIVES[self.layer_activations[l]](
                        self.Zs[l]
                    )
                )
            grads[f"dW{l}"] = (1.0 / m) * dZs[l] @ self.As[l - 1].T
            if self.regularisation_lambda and l != self.L:
                grads[f"dW{l}"] += (self.regularisation_lambda / m) * self.params[
                    f"W{l}"
                ]
            grads[f"db{l}"] = (1.0 / m) * np.sum(dZs[l], axis=1, keepdims=True)
        return grads

    def cost(self, A, Y) -> float:
        # For cost, we need to pass the weights. We'll extract them from self.params.
        Ws = {l: self.params[f"W{l}"] for l in range(1, self.L + 1)}
        if self.cost_function == "log_loss":
            return log_loss(A, Y, Ws, self.regularisation_lambda).item()
        elif self.cost_function == "square_loss":
            return square_loss(A, Y, Ws, self.regularisation_lambda).item()
        else:
            raise Exception(
                f"Incorrect value for self.cost_function:= {self.cost_function}"
            )

    def predict(self, X, return_probability=False):
        Y_hat = self.forward(X)
        if return_probability:
            return Y_hat
        return np.where(Y_hat > 0.5, 1, 0)


class Optimizer:
    """This implements gradient descent. With optional mini-batching."""

    def __init__(self, learning_rate: float = 0.1, batch_size: int | None = None):
        self.learning_rate = learning_rate
        self.batch_size = batch_size

    def update_model_params(self, model, grads, training_iteration):
        """Update model parameters using grads returned from backward."""
        for param_key in model.params:
            model.params[param_key] -= self.learning_rate * grads[f"d{param_key}"]

    def train(
        self,
        model,
        X,
        Y,
        n_epochs=1000,
        log_every: int | None = None,
        plot_cost=False,
        fig=None,
        plot_every=10,
    ):
        costs, epochs = [], []

        if plot_cost:
            if fig is None:
                fig = go.FigureWidget()
                display(fig)
            fig.add_scatter(x=[], y=[], mode="lines+markers", name=model.model_id)
            fig.update_layout(
                title="Training Cost over Epochs",
                xaxis_title="Epoch",
                yaxis_title="Cost",
            )

        m = X.shape[1]
        batch_size = self.batch_size if self.batch_size is not None else m

        training_iteration = 1
        for epoch in range(n_epochs):
            for i in range(0, m, batch_size):
                X_batch = X[:, i : i + batch_size]
                Y_batch = Y[:, i : i + batch_size]

                # Forward pass
                A = model.forward(X_batch, cache=True)
                grads = model.backward(Y_batch)
                self.update_model_params(model, grads, training_iteration)
                training_iteration += 1

            # Compute cost on the whole dataset after epoch
            A_full = model.forward(X, cache=False)
            cost = model.cost(A_full, Y)
            if log_every and epoch % log_every == 0:
                logger.info(f"Cost after epoch {epoch} = {cost}")
            if plot_cost and fig is not None:
                costs.append(cost)
                epochs.append(epoch + 1)
                if epoch % plot_every == 0:
                    with fig.batch_update():
                        fig.data[-1].x = epochs  # type: ignore
                        fig.data[-1].y = costs  # type: ignore


class MomentumOptimizer(Optimizer):
    """Implements momentum optimizer.

    The update rule is:
    v_t = beta * v_{t-1} + grad_t
    param_t = param_{t-1} - learning_rate * v_t

    Where:
    - S_t is the second moment of the gradient
    - param_t is the parameter
    - grad_t is the gradient of the cost function with respect to the parameter
    """

    def __init__(
        self,
        learning_rate: float = 0.1,
        batch_size: int | None = None,
        beta: float = 0.9,
    ):
        super().__init__(learning_rate, batch_size)
        self.beta = beta
        self.cache = {}

    def update_model_params(self, model, grads, training_iteration):
        for param_key in model.params:
            if param_key not in self.cache:
                self.cache[param_key] = np.zeros_like(model.params[param_key])
            self.cache[param_key] = (
                self.beta * self.cache[param_key]
                + (1 - self.beta) * grads[f"d{param_key}"]
            )
            model.params[param_key] -= self.learning_rate * self.cache[param_key]


class RMSPropOptimizer(Optimizer):
    """Implements RMSProp optimizer.

    Here we track the exponentially weighted average of the squared gradients (second moment).

    The update rule is:
    S_t = beta * S_{t-1} + (1 - beta) * (grad_t)^2
    param_t = param_{t-1} - learning_rate * grad_t / (sqrt(S_t) + epsilon)
    """

    def __init__(
        self,
        learning_rate: float = 0.1,
        batch_size: int | None = None,
        beta: float = 0.9,
        epsilon: float = 1e-8,
    ):
        super().__init__(learning_rate, batch_size)
        self.beta = beta
        self.epsilon = epsilon  # to avoid division by zero
        self.s_cache = {}

    def update_model_params(self, model, grads, training_iteration):
        # Standard RMSProp does NOT use bias correction (unlike Adam)
        for param_key in model.params:
            if param_key not in self.s_cache:
                self.s_cache[param_key] = np.zeros_like(model.params[param_key])
            self.s_cache[param_key] = self.beta * self.s_cache[param_key] + (
                1 - self.beta
            ) * (grads[f"d{param_key}"] ** 2)
            model.params[param_key] -= (
                self.learning_rate
                * grads[f"d{param_key}"]
                / (np.sqrt(self.s_cache[param_key]) + self.epsilon)
            )


class ADAMOptimizer(Optimizer):
    """Implements ADAM optimizer with learning rate decay.

    The update rule is:
    v_t = beta_1 * v_{t-1} + (1 - beta_1) * grad
    s_t = beta_2 * s_{t-1} + (1 - beta_2) * (grad_t)^2
    v_t_corrected = v_t / (1 - beta_1^t)
    s_t_corrected = s_t / (1 - beta_2^t)
    param_t = param_{t-1} - learning_rate * v_t_corrected / (sqrt(s_t_corrected) + epsilon)

    """

    def __init__(
        self,
        learning_rate: float = 0.1,
        batch_size: int | None = None,
        beta_1: float = 0.9,
        beta_2: float = 0.999,
        epsilon: float = 1e-8,
        learning_rate_decay: float = 0,
    ):
        super().__init__(learning_rate, batch_size)
        self.beta_1 = beta_1  # for the momentum
        self.beta_2 = beta_2  # for the second moment (RMSProp)
        self.epsilon = epsilon  # to avoid division by zero
        self.learning_rate_decay = learning_rate_decay
        self.v_cache = {}
        self.s_cache = {}

    def update_model_params(self, model, grads, training_iteration):
        learning_rate = self.learning_rate * (
            1 / (1 + self.learning_rate_decay * training_iteration)
        )
        for param_key in model.params:
            if param_key not in self.v_cache:
                self.v_cache[param_key] = np.zeros_like(model.params[param_key])
                self.s_cache[param_key] = np.zeros_like(model.params[param_key])
            self.v_cache[param_key] = (
                self.beta_1 * self.v_cache[param_key]
                + (1 - self.beta_1) * grads[f"d{param_key}"]
            )
            self.s_cache[param_key] = (
                self.beta_2 * self.s_cache[param_key]
                + (1 - self.beta_2) * grads[f"d{param_key}"] ** 2
            )
            v_t_corrected = self.v_cache[param_key] / (
                1 - self.beta_1**training_iteration
            )
            s_t_corrected = self.s_cache[param_key] / (
                1 - self.beta_2**training_iteration
            )

            model.params[param_key] -= (
                learning_rate * v_t_corrected / (np.sqrt(s_t_corrected) + self.epsilon)
            )


def plot_decision_boundary(model, X, y, resolution=0.02):
    """
    Plot the decision boundary for a model trained on all possible pairs of features.
    """
    import numpy as np
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import itertools

    n_features = X.shape[0]
    feature_names = ["sepal length", "sepal width", "petal length", "petal width"]
    class_names = ["setosa", "versicolor", "virginica"]
    colors = ["red", "blue", "green"]

    feature_pairs = list(itertools.combinations(range(n_features), 2))
    n_pairs = len(feature_pairs)
    n_rows = (n_pairs + 1) // 2
    n_cols = 2

    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        subplot_titles=[
            f"{feature_names[i]} vs {feature_names[j]}" for i, j in feature_pairs
        ],
    )

    y_classes = np.argmax(y, axis=0)
    mean_features = X.mean(axis=1)

    for idx, (feat1, feat2) in enumerate(feature_pairs):
        row = idx // 2 + 1
        col = idx % 2 + 1

        X_plot = X[[feat1, feat2], :]

        # Create mesh grid
        x_min, x_max = X_plot[0].min() - 0.5, X_plot[0].max() + 0.5
        y_min, y_max = X_plot[1].min() - 0.5, X_plot[1].max() + 0.5

        # Create mesh grid points
        x_mesh = np.linspace(x_min, x_max, 100)
        y_mesh = np.linspace(y_min, y_max, 100)
        xx, yy = np.meshgrid(x_mesh, y_mesh)

        # Prepare input for prediction
        mesh_points = np.vstack([xx.ravel(), yy.ravel()])
        X_pred = np.tile(mean_features.reshape(-1, 1), (1, mesh_points.shape[1]))
        X_pred[feat1] = mesh_points[0]
        X_pred[feat2] = mesh_points[1]

        # Get predictions
        Z_probs = model.predict(X_pred, return_probability=True)
        Z = np.argmax(Z_probs, axis=0)
        Z = Z.reshape(100, 100)

        # Plot decision boundary
        contour = fig.add_trace(
            go.Contour(
                x=x_mesh,
                y=y_mesh,
                z=Z,
                colorscale=[
                    [0, "rgb(255,0,0)"],  # red for setosa
                    [0.5, "rgb(0,0,255)"],  # blue for versicolor
                    [1, "rgb(0,255,0)"],  # green for virginica
                ],
                opacity=0.4,
                colorbar=dict(
                    title="Predicted Class",
                    ticktext=class_names,
                    tickvals=[0, 1, 2],
                    tickmode="array",
                    len=1.0,
                    yanchor="top",
                    y=1,
                )
                if idx == 0
                else None,
                contours=dict(
                    showlabels=True,
                ),
            ),
            row=row,
            col=col,
        )

        # Plot training points
        for i in range(3):
            mask = y_classes == i
            fig.add_trace(
                go.Scatter(
                    x=X_plot[0, mask],
                    y=X_plot[1, mask],
                    mode="markers",
                    name=class_names[i],
                    marker=dict(
                        size=8,
                        color=colors[i],
                    ),
                    showlegend=True if idx == 0 else False,
                ),
                row=row,
                col=col,
            )

    # Update layout
    fig.update_layout(
        title="Decision Boundaries for All Feature Pairs",
        showlegend=True,
        legend=dict(
            title="Actual Classes", yanchor="top", y=0.99, xanchor="left", x=1.05
        ),
    )

    # Update axes labels
    for idx, (feat1, feat2) in enumerate(feature_pairs):
        row = idx // 2 + 1
        col = idx % 2 + 1
        fig.update_xaxes(title_text=feature_names[feat1], row=row, col=col)
        fig.update_yaxes(title_text=feature_names[feat2], row=row, col=col)

    fig.show()


def softmax_regression_example():
    data = sns.load_dataset("iris")
    np.random.seed(45)
    # one hot encode the y values
    data = pd.get_dummies(data, columns=["species"]).astype(float)
    train_data = data.sample(frac=0.8)
    test_data = data.drop(train_data.index)
    X_train = train_data[["sepal_length", "sepal_width"]].to_numpy().T
    y_train = (
        train_data[["species_setosa", "species_versicolor", "species_virginica"]]
        .to_numpy()
        .T
    )

    X_test = test_data[["sepal_length", "sepal_width"]].to_numpy().T
    y_test = (
        test_data[["species_setosa", "species_versicolor", "species_virginica"]]
        .to_numpy()
        .T
    )

    layers = [2, 15, 5, 3]
    nn = NeuralNetwork(
        layers,
        model_id="Softmax regression",
        layer_activations={1: "relu", 2: "relu", 3: "softmax"},
    )
    nn.initialise_weights()
    adam_optimizer = ADAMOptimizer(batch_size=128, learning_rate=0.01)

    fig = go.FigureWidget()
    display(fig)

    from time import perf_counter

    start_time = perf_counter()
    adam_optimizer.train(nn, X_train, y_train, n_epochs=500, plot_cost=True, fig=fig)
    end_time = perf_counter()
    time = end_time - start_time
    print(f"\n Optimiser: {adam_optimizer}")
    print(f"Training time: {time:.4f} seconds")
    # Get predictions
    train_Y_pred = nn.predict(X_train)
    test_Y_pred = nn.predict(X_test)

    # Convert one-hot encoded targets to class indices
    y_train_classes = np.argmax(y_train, axis=0)
    y_test_classes = np.argmax(y_test, axis=0)

    train_Y_pred_classes = np.argmax(train_Y_pred, axis=0)
    test_Y_pred_classes = np.argmax(test_Y_pred, axis=0)

    print(f"train_Y_pred_classes: {train_Y_pred_classes}")

    # Calculate accuracies using class indices
    train_accuracy = (train_Y_pred_classes == y_train_classes).mean()
    test_accuracy = (test_Y_pred_classes == y_test_classes).mean()

    print(f"Accuracy on train: {train_accuracy}")
    print(f"Accuracy on test: {test_accuracy}")
    print(f"Accuracy on train: {train_accuracy}")
    print(f"Accuracy on test: {test_accuracy}")

    # plot the decision boundary
    plot_decision_boundary(nn, X_train, y_train)


softmax_regression_example()

FigureWidget({
    'data': [], 'layout': {'template': '...'}
})


 Optimiser: <__main__.ADAMOptimizer object at 0x1394624d0>
Training time: 0.1505 seconds
train_Y_pred_classes: [0 0 2 0 0 0 0 2 2 2 0 2 1 2 1 0 2 1 0 2 1 1 2 2 0 2 1 1 0 1 2 1 1 1 0 2 0
 0 2 0 0 1 0 2 1 0 2 1 2 0 0 1 2 0 2 0 0 1 0 1 2 1 1 2 0 0 0 0 1 0 2 2 1 1
 1 0 0 1 0 1 2 2 1 1 1 2 1 0 1 1 1 2 1 1 2 1 2 1 1 2 0 0 1 2 2 1 0 2 2 1 2
 2 0 2 2 0 0 2 0 2]
Accuracy on train: 0.825
Accuracy on test: 0.8
Accuracy on train: 0.825
Accuracy on test: 0.8


### Batch Normalisation



- Batch normalisation is a technique that normalises the inputs of each layer. The parameters $\gamma$ and $\beta$ are learnable parameters that scale and shift the normalised inputs. the values of $\mu$ and $\sigma^2$ are calculated for each mini-batch.
$$
 \begin{align*}
 \mu &= \frac{1}{m} \sum_{i=1}^{m} z^{(i)} \\
 \sigma^2 &= \frac{1}{m} \sum_{i=1}^{m} (z^{(i)} - \mu)^2 \\
 z^{(i)}_{norm} &= \frac{z^{(i)} - \mu}{\sqrt{\sigma^2 + \epsilon}} \\
 \tilde{z}^{(i)} &= \gamma z^{(i)}_{norm} + \beta
 \end{align*}
 $$
 - We also keep track of the exponentially weighted average of the mean and variance to use during prediction.
 $$
 \begin{align*}
 \mu &= \beta_1 \mu_{t-1} + (1 - \beta_1) \mu_t \\
 \sigma^2 &= \beta_2 \sigma_{t-1}^2 + (1 - \beta_2) \sigma_t^2
 \end{align*}
 $$
 - Now the - The forward pass is calculated as
$$
\begin{align}
Z^{[l]} &= W^{[l]}A^{[l-1]}+b^{[l]} \\
\mu^{[l]} &= Z^{(i)}.mean(axis=1) \\
{\sigma^{[l]}}^2 &= (z^{(i)} - \mu)^2.mean(axis=1) \\
Z^{(i)}_{norm} &= \frac{Z^{(i)} - \mu}{\sqrt{\sigma^2 + \epsilon}} \\
A^{[l]} &= g(Z^{(i)}_{norm}), \quad\text{lth layer, g - activation function} \\ 
Z^{[0]} &= X \\
\hat{Y} &= A^{[L]} \\
J &= -\frac{1}{m}(Ylog(A)^T + (1-Y)log(1-A)^T) \\
\end{align}
$$


See `neural_network_with_bn(.md|.py)`


### Tensorflow

In [9]:
import tensorflow as tf

# This command lists all available physical devices (CPUs and GPUs)
gpu_devices = tf.config.list_physical_devices('GPU')

if gpu_devices:
    print(f"✅ Success! TensorFlow has found your Mac's GPU:")
    for device in gpu_devices:
        print(f"- {device}")
else:
    print("❌ TensorFlow did not find the GPU. It will run on the CPU instead.")

✅ Success! TensorFlow has found your Mac's GPU:
- PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [ ]:
import tensorflow as tf


def simple_tensor_flow_example():
    # in this example we will minimise x**2 - 5x + 6

    x = tf.Variable(0.0, name="x", dtype=tf.float32)

    def loss():
        return x**2 - 5 * x + 6

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
    n = 100
    for _ in range(n):
        with tf.GradientTape() as tape:
            loss_value = loss()
        gradients = tape.gradient(loss_value, [x])
        optimizer.apply_gradients(zip(gradients, [x]))

    print(f"After {n} iterations, x = {x.numpy()}")


simple_tensor_flow_example()

2025-10-01 09:26:30.402362: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Max
2025-10-01 09:26:30.402395: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2025-10-01 09:26:30.402400: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2025-10-01 09:26:30.402430: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-10-01 09:26:30.402439: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


After 100 iterations, x = 2.5024521350860596
